# 🌾 Precision Farming: Soil & Environmental Data Clustering

This notebook performs multivariate clustering analysis on soil and environmental data to support precision farming decisions.

**Features:**
- Multiple clustering algorithms (K-Means, DBSCAN, Hierarchical, GMM)
- PCA dimensionality reduction
- Interactive visualizations
- Cluster profiling and statistics
- Downloadable results

---

## 📦 Step 1: Install Required Packages

In [ ]:
!pip install pandas numpy scikit-learn plotly matplotlib seaborn openpyxl -q

## 📚 Step 2: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

from google.colab import files
import io
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

## 🔧 Step 3: Backend Clustering Engine

In [ ]:
class SoilClusteringEngine:
    def __init__(self):
        self.scaler = StandardScaler()
        self.pca = None
        self.models = {}
        self.scaled_data = None
        self.pca_data = None
        self.results = {}
        self.feature_names = None
        
    def preprocess_data(self, df, feature_columns):
        """Standardize the data"""
        self.feature_names = feature_columns
        data = df[feature_columns].values
        self.scaled_data = self.scaler.fit_transform(data)
        return self.scaled_data
    
    def apply_pca(self, n_components=3):
        """Apply PCA for dimensionality reduction"""
        if self.scaled_data is None:
            raise ValueError("Data must be preprocessed first")
        
        self.pca = PCA(n_components=n_components)
        self.pca_data = self.pca.fit_transform(self.scaled_data)
        
        return {
            'pca_data': self.pca_data,
            'variance_explained': self.pca.explained_variance_ratio_,
            'cumulative_variance': np.cumsum(self.pca.explained_variance_ratio_)
        }
    
    def calculate_elbow_scores(self, max_clusters=10):
        """Calculate elbow metrics for K-Means"""
        inertias = []
        silhouette_scores = []
        davies_bouldin_scores = []
        k_range = range(2, min(max_clusters + 1, len(self.scaled_data)))
        
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(self.scaled_data)
            
            inertias.append(kmeans.inertia_)
            silhouette_scores.append(silhouette_score(self.scaled_data, labels))
            davies_bouldin_scores.append(davies_bouldin_score(self.scaled_data, labels))
        
        return {
            'k_values': list(k_range),
            'inertias': inertias,
            'silhouette_scores': silhouette_scores,
            'davies_bouldin_scores': davies_bouldin_scores
        }
    
    def perform_kmeans(self, n_clusters):
        """Perform K-Means clustering"""
        model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = model.fit_predict(self.scaled_data)
        
        return self._get_metrics('K-Means', model, labels)
    
    def perform_dbscan(self, eps=0.5, min_samples=5):
        """Perform DBSCAN clustering"""
        model = DBSCAN(eps=eps, min_samples=min_samples)
        labels = model.fit_predict(self.scaled_data)
        
        return self._get_metrics('DBSCAN', model, labels)
    
    def perform_hierarchical(self, n_clusters, linkage='ward'):
        """Perform Hierarchical clustering"""
        model = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
        labels = model.fit_predict(self.scaled_data)
        
        return self._get_metrics('Hierarchical', model, labels)
    
    def perform_gmm(self, n_components, covariance_type='full'):
        """Perform Gaussian Mixture Model clustering"""
        model = GaussianMixture(n_components=n_components, covariance_type=covariance_type, random_state=42)
        model.fit(self.scaled_data)
        labels = model.predict(self.scaled_data)
        
        result = self._get_metrics('GMM', model, labels)
        result['bic'] = model.bic(self.scaled_data)
        result['aic'] = model.aic(self.scaled_data)
        return result
    
    def _get_metrics(self, algorithm_name, model, labels):
        """Calculate clustering metrics"""
        unique_labels = set(labels)
        n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
        
        result = {
            'algorithm': algorithm_name,
            'labels': labels,
            'model': model,
            'n_clusters': n_clusters
        }
        
        if n_clusters > 1 and len(unique_labels) > 1:
            valid_mask = labels != -1
            if valid_mask.sum() > 0:
                result['silhouette_score'] = silhouette_score(self.scaled_data[valid_mask], labels[valid_mask])
                result['davies_bouldin_score'] = davies_bouldin_score(self.scaled_data[valid_mask], labels[valid_mask])
                result['calinski_harabasz_score'] = calinski_harabasz_score(self.scaled_data[valid_mask], labels[valid_mask])
        
        return result
    
    def get_cluster_statistics(self, df, feature_columns, labels):
        """Calculate cluster statistics"""
        df_with_clusters = df.copy()
        df_with_clusters['Cluster'] = labels
        
        cluster_stats = df_with_clusters.groupby('Cluster')[feature_columns].agg(['mean', 'std', 'min', 'max']).round(3)
        cluster_sizes = df_with_clusters['Cluster'].value_counts().sort_index()
        
        return {
            'statistics': cluster_stats,
            'sizes': cluster_sizes,
            'df_with_clusters': df_with_clusters
        }

print("✅ Clustering engine class defined!")

## 📂 Step 4: Upload Your CSV File

In [ ]:
print("Please upload your soil/environmental data CSV file:")
uploaded = files.upload()

# Load the uploaded file
filename = list(uploaded.keys())[0]
if filename.endswith('.csv'):
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
else:
    df = pd.read_excel(io.BytesIO(uploaded[filename]))

print(f"\n✅ Loaded {len(df)} rows and {len(df.columns)} columns")
print("\n📊 Data Preview:")
display(df.head())
print("\n📈 Statistical Summary:")
display(df.describe())

## 🔍 Step 5: Data Exploration

In [ ]:
# Show all columns
print("Available columns in your dataset:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col} (Type: {df[col].dtype})")

# Get numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\n🔢 Found {len(numeric_columns)} numeric columns for clustering")

# Correlation heatmap
if len(numeric_columns) > 1:
    plt.figure(figsize=(12, 8))
    sns.heatmap(df[numeric_columns].corr(), annot=True, cmap='RdBu', center=0, fmt='.2f')
    plt.title('Feature Correlation Heatmap')
    plt.tight_layout()
    plt.show()

## ✅ Step 6: Select Features for Clustering

Modify the `selected_features` list below with the column names you want to use for clustering.

In [ ]:
# MODIFY THIS: Select features for clustering
# Example: selected_features = ['pH', 'Nitrogen_ppm', 'Phosphorus_ppm', 'Potassium_ppm']
selected_features = numeric_columns[:min(8, len(numeric_columns))]  # Auto-select first 8 numeric columns

print(f"Selected features for clustering: {selected_features}")
print(f"Total features: {len(selected_features)}")

## 🔧 Step 7: Preprocessing and PCA

In [ ]:
# Initialize clustering engine
engine = SoilClusteringEngine()

# Preprocess data
scaled_data = engine.preprocess_data(df, selected_features)
print("✅ Data standardized")

# Apply PCA
n_components = min(3, len(selected_features))
pca_result = engine.apply_pca(n_components=n_components)

print(f"\n📊 PCA Results:")
print(f"Components: {n_components}")
print(f"Variance explained by each component: {[f'{v*100:.2f}%' for v in pca_result['variance_explained']]}")
print(f"Total variance explained: {pca_result['cumulative_variance'][-1]*100:.2f}%")

# Visualize variance explained
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.bar(range(1, n_components+1), pca_result['variance_explained']*100)
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Variance Explained (%)')
ax1.set_title('Variance Explained by Each Component')

ax2.plot(range(1, n_components+1), pca_result['cumulative_variance']*100, marker='o')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Variance (%)')
ax2.set_title('Cumulative Variance Explained')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📊 Step 8: Elbow Method (K-Means Optimization)

In [ ]:
# Calculate elbow metrics
elbow_data = engine.calculate_elbow_scores(max_clusters=10)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(elbow_data['k_values'], elbow_data['inertias'], marker='o', linewidth=2)
axes[0].set_xlabel('Number of Clusters (k)')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Elbow Method (Inertia)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(elbow_data['k_values'], elbow_data['silhouette_scores'], marker='o', linewidth=2, color='green')
axes[1].set_xlabel('Number of Clusters (k)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score (Higher is Better)')
axes[1].grid(True, alpha=0.3)

axes[2].plot(elbow_data['k_values'], elbow_data['davies_bouldin_scores'], marker='o', linewidth=2, color='red')
axes[2].set_xlabel('Number of Clusters (k)')
axes[2].set_ylabel('Davies-Bouldin Index')
axes[2].set_title('Davies-Bouldin Index (Lower is Better)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Suggest optimal k
best_k_idx = np.argmax(elbow_data['silhouette_scores'])
suggested_k = elbow_data['k_values'][best_k_idx]
print(f"\n💡 Suggested optimal clusters: {suggested_k} (based on Silhouette Score)")

## 🎯 Step 9: Run Clustering Algorithms

Choose which algorithm(s) to run by setting the parameters below.

In [ ]:
# MODIFY THESE PARAMETERS
n_clusters = 3  # For K-Means, Hierarchical, GMM

# Run all algorithms
results = {}

print("Running clustering algorithms...\n")

# 1. K-Means
print("1️⃣ K-Means...")
results['K-Means'] = engine.perform_kmeans(n_clusters=n_clusters)
print(f"   Clusters: {results['K-Means']['n_clusters']}")
print(f"   Silhouette: {results['K-Means'].get('silhouette_score', 'N/A')}")

# 2. DBSCAN
print("\n2️⃣ DBSCAN...")
results['DBSCAN'] = engine.perform_dbscan(eps=0.8, min_samples=5)
print(f"   Clusters: {results['DBSCAN']['n_clusters']}")
print(f"   Silhouette: {results['DBSCAN'].get('silhouette_score', 'N/A')}")

# 3. Hierarchical
print("\n3️⃣ Hierarchical...")
results['Hierarchical'] = engine.perform_hierarchical(n_clusters=n_clusters, linkage='ward')
print(f"   Clusters: {results['Hierarchical']['n_clusters']}")
print(f"   Silhouette: {results['Hierarchical'].get('silhouette_score', 'N/A')}")

# 4. Gaussian Mixture Model
print("\n4️⃣ Gaussian Mixture Model...")
results['GMM'] = engine.perform_gmm(n_components=n_clusters, covariance_type='full')
print(f"   Clusters: {results['GMM']['n_clusters']}")
print(f"   Silhouette: {results['GMM'].get('silhouette_score', 'N/A')}")
print(f"   BIC: {results['GMM'].get('bic', 'N/A')}")

print("\n✅ All algorithms completed!")

## 📊 Step 10: Algorithm Comparison

In [ ]:
# Create comparison table
comparison_data = []
for algo_name, result in results.items():
    row = {
        'Algorithm': algo_name,
        'Clusters': result['n_clusters'],
        'Silhouette': round(result.get('silhouette_score', 0), 4),
        'Davies-Bouldin': round(result.get('davies_bouldin_score', 0), 4),
        'Calinski-Harabasz': round(result.get('calinski_harabasz_score', 0), 2)
    }
    if 'bic' in result:
        row['BIC'] = round(result['bic'], 2)
        row['AIC'] = round(result['aic'], 2)
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
print("\n📊 Algorithm Comparison:")
display(comparison_df)

print("\n💡 Metric Interpretation:")
print("  • Silhouette Score: Higher is better (range: -1 to 1)")
print("  • Davies-Bouldin Index: Lower is better")
print("  • Calinski-Harabasz: Higher is better")
print("  • BIC/AIC (GMM only): Lower is better")

## 📈 Step 11: Visualize Clustering Results

In [ ]:
# 3D Visualization for each algorithm
if pca_result['pca_data'].shape[1] >= 3:
    for algo_name, result in results.items():
        df_plot = pd.DataFrame({
            'PC1': pca_result['pca_data'][:, 0],
            'PC2': pca_result['pca_data'][:, 1],
            'PC3': pca_result['pca_data'][:, 2],
            'Cluster': result['labels'].astype(str)
        })
        
        fig = px.scatter_3d(
            df_plot,
            x='PC1', y='PC2', z='PC3',
            color='Cluster',
            title=f'{algo_name} - 3D Cluster Visualization',
            labels={'PC1': 'PC1', 'PC2': 'PC2', 'PC3': 'PC3'}
        )
        fig.update_traces(marker=dict(size=5))
        fig.show()
else:
    print("Not enough components for 3D visualization")

## 📉 2D Visualizations

In [ ]:
# 2D scatter plots for all algorithms
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, (algo_name, result) in enumerate(results.items()):
    scatter = axes[idx].scatter(
        pca_result['pca_data'][:, 0],
        pca_result['pca_data'][:, 1],
        c=result['labels'],
        cmap='viridis',
        s=50,
        alpha=0.6,
        edgecolors='white',
        linewidth=0.5
    )
    axes[idx].set_xlabel('Principal Component 1')
    axes[idx].set_ylabel('Principal Component 2')
    axes[idx].set_title(f'{algo_name} (n={result["n_clusters"]} clusters)')
    axes[idx].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[idx], label='Cluster')

plt.tight_layout()
plt.show()

## 📋 Step 12: Cluster Statistics and Profiling

Choose which algorithm's results to analyze in detail:

In [ ]:
# MODIFY THIS: Choose algorithm for detailed analysis
selected_algorithm = 'K-Means'  # Options: 'K-Means', 'DBSCAN', 'Hierarchical', 'GMM'

selected_result = results[selected_algorithm]
cluster_stats = engine.get_cluster_statistics(df, selected_features, selected_result['labels'])

print(f"\n📊 Detailed Analysis for {selected_algorithm}\n")

# Cluster sizes
print("Cluster Sizes:")
for cluster_id, size in cluster_stats['sizes'].items():
    percentage = (size / len(df) * 100)
    print(f"  Cluster {cluster_id}: {size} samples ({percentage:.1f}%)")

# Cluster statistics
print("\nCluster Statistics (Mean Values):")
display(cluster_stats['statistics'].xs('mean', axis=1, level=1))

# Radar chart for cluster comparison
cluster_means = cluster_stats['statistics'].xs('mean', axis=1, level=1)
fig = go.Figure()

for cluster in cluster_means.index:
    fig.add_trace(go.Scatterpolar(
        r=cluster_means.loc[cluster].values,
        theta=selected_features,
        fill='toself',
        name=f'Cluster {cluster}'
    ))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True)),
    showlegend=True,
    title=f"Cluster Profile Comparison - {selected_algorithm}",
    height=600
)
fig.show()

## 💾 Step 13: Download Results

In [ ]:
# Save clustered data
df_export = cluster_stats['df_with_clusters']
df_export.to_csv('clustered_soil_data.csv', index=False)
print("✅ Saved: clustered_soil_data.csv")

# Save cluster statistics
cluster_stats['statistics'].to_csv('cluster_statistics.csv')
print("✅ Saved: cluster_statistics.csv")

# Save comparison
comparison_df.to_csv('algorithm_comparison.csv', index=False)
print("✅ Saved: algorithm_comparison.csv")

# Download files
print("\n📥 Downloading files...")
files.download('clustered_soil_data.csv')
files.download('cluster_statistics.csv')
files.download('algorithm_comparison.csv')

print("\n✅ All files downloaded successfully!")

## 📝 Analysis Summary

In [ ]:
print("="*60)
print("PRECISION FARMING CLUSTERING ANALYSIS REPORT")
print("="*60)
print(f"\nDataset: {len(df)} samples, {len(selected_features)} features")
print(f"Features analyzed: {', '.join(selected_features)}")
print(f"\nPCA: {n_components} components explaining {pca_result['cumulative_variance'][-1]*100:.1f}% variance")
print("\nAlgorithm Results:")
for algo_name, result in results.items():
    print(f"\n  {algo_name}:")
    print(f"    - Clusters: {result['n_clusters']}")
    if 'silhouette_score' in result:
        print(f"    - Silhouette Score: {result['silhouette_score']:.4f}")
    if 'davies_bouldin_score' in result:
        print(f"    - Davies-Bouldin: {result['davies_bouldin_score']:.4f}")

print("\n" + "="*60)
print("Analysis complete! Check the downloaded CSV files for detailed results.")
print("="*60)